# Radio Map

In [3]:
import pickle 
import json
import pandas as pd
import numpy as np

In [4]:
with open('RPs.pkl' , 'rb') as fp:
    d = pickle.load(fp)

In [5]:
with open('meta.json' , 'r') as fp:
    meta = json.load(fp)

In [6]:
'''
d has the following structure -> {location : RSSI value}


location - > room number.location in that room.timestamp.random
RSSI value -> a vector of length len(AP_LIST)
'''

'\nd has the following structure -> {location : RSSI value}\n\n\nlocation - > room number.location in that room.timestamp.random\nRSSI value -> a vector of length len(AP_LIST)\n'

In [39]:
AP_LIST = meta['ap_list']
AP_MAP = {AP_LIST[i] : i for i in range(len(AP_LIST))}
NUM_RECORDS = len(d)

In [8]:
df = pd.DataFrame(AP_LIST, columns = ['AP'])
temp = pd.DataFrame(d.values()).transpose()
temp.columns = d.keys()
df = pd.concat([df, temp] , axis = 1)
df = df.set_index('AP')

In [9]:
# dd = pd.DataFrame(AP_LIST, columns = ['AP'])
# for k in list(d.keys()):
#     temp = pd.DataFrame(d[k], columns=[k])
#     dd = pd.concat([dd, temp] , axis = 1)
# dd = dd.set_index('AP')

In [10]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [11]:
RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
TIMESTEPS = 5

In [12]:
radio_map = {}

for direction in ['north', 'south', 'east', 'west']:
    arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
    for key in RP_MAP:
        cols = [x for x in df.columns if key + '.' + direction in x]
        temp = df[cols].to_numpy()
        arr[:,RP_MAP[key],:] = temp 

    radio_map[direction] = arr


In [16]:
radio_map['north'].shape

(953, 116, 5)

# Reliability and Similarity

In [45]:
GAMMA = -80  #Threshold aboce whcih an AP readig is too small
M = 3 #How many of the 5 time readings does the AP value need to be above GAMMA
LAMBDA = 1 / 1000 #Specifies a small amount to be added to hamming distance denominator to avoid zero division error

In [35]:
I = {} #Indicator matrices to check for "reliable" APs

In [36]:
for direction in ['north', 'south', 'east', 'west']:
    T = (radio_map[direction] > GAMMA).astype(int).sum(axis = 2)
    _I = (T > M).astype(int)
    I[direction] = _I

In [46]:
S = {} #Similarity matrices which stores a score for how "close" two RPs are

In [47]:
def hamming(a, b):
    return 1 / (np.count_nonzero(a!=b) + LAMBDA)

In [48]:
for direction in ['north', 'south', 'east', 'west']:
    _S = np.ndarray((len(RP_MAP) , len(RP_MAP)))
    _I = I[direction]
    for i in range(len(RP_MAP)):
        for j in range(len(RP_MAP)):
            _S[i][j] = hamming(_I[:, i] , _I[:, j])
    S[direction] = _S

# Stability